In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error
import csv
import matplotlib.pyplot as plt
import seaborn as sns


import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [61]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.executemany(sql_statement)

    rows = cur.fetchall()

    return rows

In [3]:
filename = "fake_job_postings.csv"

In [5]:
with open(filename) as file:
    raw_data = list(csv.reader(file, delimiter=","))
raw_data = raw_data[1:]
raw_data1 = [tuple(i) for i in raw_data]

In [57]:
job_idx = []
titles = []
depts = []
salaries = []
requirements = []
benefits = []

for data in raw_data:
    job_id = data[0]
    job_idx.append(job_id)
    
    title = data[1]
    titles.append((title,))
    
    dept = data[3]
    depts.append((dept,))
    
    salary_range = data[4]
    if '-' in salary_range:
        min_salary = salary_range.split('-')[0].split(' ')[0]
        max_salary = salary_range.split('-')[1].split(' ')[-1]
    else:
        min_salary = ''
        max_salary = ''
    salaries.append((salary_range, min_salary, max_salary))
    
    requirement = data[6]
    requirements.append((requirement,))
    
    benefit = data[7]
    benefits.append((benefit,))

titles = tuple(set(titles))
depts = tuple(set(depts))
salaries = tuple(set(salaries))
requirements = tuple(set(requirements))
benefits = tuple(set(benefits))

In [68]:
def insert_titles(titles):
    conn = create_connection('mubin.db')
    table_sql = """
                    CREATE TABLE [Titles] (
                        [TitleID] Integer not null primary key,
                        [Title] Text
                    )
                    """
    create_table(conn, table_sql, "Titles")
    cur = conn.cursor()
    insert_sql = """
                    INSERT INTO Titles (Title)
                    Values (?)
                    """
    cur.executemany(insert_sql, titles)
    conn.commit()
    cur.close()
    conn.close()
    
def insert_departments(departments):
    conn = create_connection('mubin.db')
    table_sql = """
                    CREATE TABLE [Departments] (
                        [DepartmentID] Integer not null primary key,
                        [Department] Text
                    )
                    """
    create_table(conn, table_sql, "Departments")
    cur = conn.cursor()
    insert_sql = """
                    INSERT INTO Departments (Department)
                    Values (?)
                    """
    cur.executemany(insert_sql, departments)
    conn.commit()
    cur.close()
    conn.close()
    
def insert_salaries(salaries):
    conn = create_connection('mubin.db')
    table_sql = """
                    CREATE TABLE [Salaries] (
                        [SalaryID] Integer not null primary key,
                        [Salary_Range] Text,
                        [Min_Salary] REAL,
                        [Max_Salary] REAL
                    );
                    """
    create_table(conn, table_sql, "Salaries")
    cur = conn.cursor()
    insert_sql = """
                    INSERT INTO Salaries (Salary_Range, Min_Salary, Max_Salary)
                    Values (?,?,?)
                    """
    cur.executemany(insert_sql, salaries)
    conn.commit()
    cur.close()
    conn.close()
    
def insert_requirements(requirements):
    conn = create_connection('mubin.db')
    table_sql = """
                    CREATE TABLE [Requirements] (
                        [RequirementID] Integer not null primary key,
                        [Requirement] Text
                    )
                    """
    create_table(conn, table_sql, "Requirements")
    cur = conn.cursor()
    insert_sql = """
                    INSERT INTO Requirements (Requirement)
                    Values (?)
                    """
    cur.executemany(insert_sql, requirements)
    conn.commit()
    cur.close()
    conn.close()
    
def insert_benefits(benefits):
    conn = create_connection('mubin.db')
    table_sql = """
                    CREATE TABLE [Benefits] (
                        [BenefitID] Integer not null primary key,
                        [Benefit] Text
                    )
                    """
    create_table(conn, table_sql, "Benefits")
    cur = conn.cursor()
    insert_sql = """
                    INSERT INTO Benefits (Benefit)
                    Values (?)
                    """
    cur.executemany(insert_sql, benefits)
    conn.commit()
    cur.close()
    conn.close()
    
insert_titles(titles)
insert_departments(depts)
insert_salaries(salaries)
insert_requirements(requirements)
insert_benefits(benefits)

In [63]:
insert_titles(titles)

In [65]:
def insert_departments(departments):
    conn = create_connection('mubin.db')
    table_sql = """
                    CREATE TABLE [Departments] (
                        [DepartmentID] Integer not null primary key,
                        [Department] Text
                    )
                    """
    create_table(conn, table_sql)
    cur = conn.cursor()
    insert_sql = """
                    INSERT INTO Departments (Department)
                    Values (?)
                    """
    cur.executemany(insert_sql, departments)
    conn.commit()
    cur.close()
    conn.close()

In [44]:
len(benefits)

17880

In [45]:
len(tuple(set(benefits)))

11970

In [24]:
'2000 - 2800'.split('-')[1].split(' ')

['', '2800']

In [94]:
# raw_data = []
# with open(filename) as file:
#     for line in file:
#         if not line.strip():
#             continue
#         raw_data.append(line.strip())

# lst = [i.split(',') for i in raw_data]
# lst = lst[1:]
# lst1 = [tuple(i) for i in lst]

In [16]:
db_file = 'raw_data.db'
import os
if os.path.exists(db_file):
    os.remove(db_file)
conn = create_connection(db_file)

create_table_sql1 = """
    CREATE TABLE IF NOT EXISTS [raw_data] (
        [job_id] integer not null primary key,
        [title] text,
        [location] text,
        [department] text,
        [salary_range] text,
        [company_profile] text,
        [description] text,
        [requirements] text,
        [benefits] text,
        [telecommuting] integer,
        [has_company_logo] integer,
        [has_questions] integer,
        [employment_type] text,
        [required_experience] text,
        [required_education] text,
        [industry] text,
        [function] text,
        [fraudulent] integer
    );
    """

def insert_raw_data(conn, values):
    sql = ''' INSERT INTO raw_data(job_id,title,location,
                department,salary_range,company_profile,description,
                requirements,benefits,telecommuting,has_company_logo,
                has_questions,employment_type,required_experience,
                required_education,industry,function,fraudulent) 
                VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

with conn:
        create_table(conn, create_table_sql1)
        for values in raw_data1:
            insert_raw_data(conn,values)

In [17]:
db_file_norm = 'normalized_data.db'
if os.path.exists(db_file_norm):
    os.remove(db_file_norm)
conn1 = create_connection(db_file_norm)

In [22]:
with conn:
    sql_statement = 'SELECT distinct company_profile from raw_data'
    value_profile = execute_sql_statement(sql_statement, conn)

create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [company] (
        [companyID] integer not null primary key,
        [company_profile] text
    );
    """

def insert_profile(conn, values):
    sql = ''' INSERT INTO company(company_profile) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2)
    for values in value_profile:
        insert_profile(conn1,values)

In [ ]:
with conn:
    sql_statement = 'SELECT distinct company_profile from raw_data'
    value_profile = execute_sql_statement(sql_statement, conn)

create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [company] (
        [companyID] integer not null primary key,
        [company_profile] text
    );
    """

def insert_profile(conn, values):
    sql = ''' INSERT INTO company(company_profile) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2)
    for values in value_profile:
        insert_profile(conn1,values)

In [27]:
sql_statement = 'SELECT distinct company_profile, has_company_logo,count(job_id) from raw_data group by company_profile,has_company_logo order by company_profile'
df = pd.read_sql_query(sql_statement, conn)
display(df)

,company_profile,has_company_logo,count(job_id)
0,,0,2622
1,,1,686
2,Value Added Team of Creative ProfessionalsNet...,1,1
3,"""Only stupid questions create wealth"" - Gary ...",1,1
4,"""Our mission to our clients is to preserve the...",1,64
...,...,...,...
1706,Το #URL_bb79581a561837ad604b8fc4ab629753b36407...,1,4
1707,"УУРРАА - технологическая компания, работающая ...",1,9
1708,“News360 is Changing the Content Delivery Game...,1,1
1709,"“No surveys, social data can answer your quest...",1,1


In [36]:
l1 = df.iloc[2:,0].values
l1.shape

(1709,)

In [39]:
np.unique(l1).shape

(1709,)

In [43]:
sql_statement = 'SELECT distinct department, count(job_id) from raw_data group by department'
df = pd.read_sql_query(sql_statement, conn)
df.head(50)

,department,count(job_id)
0,,11547
1,,6
2,\tCorporate Shared Services,1
3,Lower Level Management,1
4,Marketing,1
5,Moni Technologies,1
6,R&D,1
7,(Consultant),1
8,.NET,1
9,.net Development,1


- title - title
- department - distinct department
- salary - id, distinct salary(min max)
- description - description
- requirements - id, req
- benefits - id,benefits
- telecommuting
- has_company_logo
- has_questions
- employment_type - 
- required_experience
- required_education
- industry
- function
- fraudulent
- job_title_loc: job_id, title, locid, titleid, salaryid
- company table: companyid, company_profile, has_company_logo
- location table: locid,location - seggregated
- jobid - companyid,  ###company_profile
- 
- posting: jobid, description, requirement, benefits, telecommuting, has_questions,employment_type,required_experience,required_education,industry,function
- 

In [ ]:
with conn:
    sql_statement = 'SELECT distinct title from raw_data'
    value_profile = execute_sql_statement(sql_statement, conn)

create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [title] (
        [title] text not null primary key
    );
    """

def insert_profile(conn, values):
    sql = ''' INSERT INTO company(company_profile) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2)
    for values in value_profile:
        insert_profile(conn1,values)

In [3]:
# df = pd.read_csv("fake_job_postings.csv")

In [112]:
# df.head()

In [6]:
# df.shape

(17880, 18)